Environment Check

Verify that we're using the correct Python environment and that all packages are available.


# Snow Elastic Modulus Calculations - Köchle and Schneebeli (2014) Method

This notebook demonstrates the application of the Köchle and Schneebeli (2014) elastic modulus parameterization to snowpit data. The Köchle method calculates Young's modulus from snow density using exponential relationships fitted from X-ray microcomputer tomography (m-CT) and finite-element (FE) simulations.

The analysis uses the local snowpyt_mechparams package and snowpylot for CAAML parsing.


In [1]:
# Import Libraries
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from uncertainties import ufloat
import warnings


# Add the src directory to the path to import snowpyt_mechparams
sys.path.append('../src')
from snowpilot_utils import convert_grain_form, parse_sample_pits
from snowpyt_mechparams import density, elastic_modulus


Parse Snowpit Files


In [2]:
# Parse all snowpit files from the data folder
all_pits = parse_sample_pits('data')


Successfully parsed 50278 files
Failed to parse 0 files


Apply Köchle parameterization using density and associated uncertainty from:
1. Direct Measurement
2. Kim_geldsetzer

In [3]:
# Collect relevant data from each snowpit
pit_info = []
layer_info = []
density_info = []  # New list for density observations

for pit in all_pits:
    pit_dict = {
        'pit_id': pit.core_info.pit_id,
        'layer_count': len(pit.snow_profile.layers),
    }
    pit_info.append(pit_dict)

    for layer in pit.snow_profile.layers:
        # Create base layer dictionary
        layer_depth_top = layer.depth_top[0] if layer.depth_top else None
        layer_thickness = layer.thickness[0] if layer.thickness else None
        
        layer_dict = {
            'pit_id': pit.core_info.pit_id,
            'hand_hardness': layer.hardness,
            'depth_top': layer_depth_top,      # Add for merging (scalar)
            'thickness': layer_thickness,      # Add for merging (scalar)
        }

        # Add kim_jamieson_table2 grain form conversion if grain form data exists
        if layer.grain_form_primary:
            layer_dict['kim_jamieson_table2_grain_form'] = convert_grain_form(layer.grain_form_primary, 'kim_jamieson_table2')
            try: 
                # Calculate density using kim_jamieson_table2 method - only if we have valid inputs
                if layer.hardness and layer_dict['kim_jamieson_table2_grain_form']:
                    density_ufloat = density.calculate_density( 
                        method='kim_jamieson_table2',
                        hand_hardness=layer.hardness,
                        grain_form=layer_dict['kim_jamieson_table2_grain_form']
                    )
                    layer_dict['density_kim_jamieson_table2'] = density_ufloat.nominal_value
                    layer_dict['density_kim_jamieson_table2_uncertainty'] = density_ufloat.std_dev
                else:
                    layer_dict['density_kim_jamieson_table2'] = None
                    layer_dict['density_kim_jamieson_table2_uncertainty'] = None
            except Exception:
                layer_dict['density_kim_jamieson_table2'] = None
                layer_dict['density_kim_jamieson_table2_uncertainty'] = None
        else:
            layer_dict['kim_jamieson_table2_grain_form'] = None
            layer_dict['density_kim_jamieson_table2'] = None
            layer_dict['density_kim_jamieson_table2_uncertainty'] = None

        layer_info.append(layer_dict)

    # Collect density observations separately
    for density_obs in pit.snow_profile.density_profile:
        # Extract scalar density value
        density_value = density_obs.density
        if hasattr(density_value, '__len__') and len(density_value) > 0:
            density_value = density_value[0]  # Take first element if it's an array
        if hasattr(density_value, 'nominal_value'):
            density_value = density_value.nominal_value  # Extract nominal value if it's a ufloat
        
        obs_depth_top = density_obs.depth_top[0] if density_obs.depth_top else None
        obs_thickness = density_obs.thickness[0] if density_obs.thickness else None
        
        density_dict = {
            'pit_id': pit.core_info.pit_id,
            'depth_top': obs_depth_top,
            'thickness': obs_thickness,
            'density_measured': float(density_value) if density_value is not None else None
        }
        density_info.append(density_dict)

# Create dataframes
pit_df = pd.DataFrame(pit_info)
layer_df = pd.DataFrame(layer_info)
density_df = pd.DataFrame(density_info)

# Merge density_measured into layer_df using pandas merge (efficient matching)
layer_df = layer_df.merge(
    density_df[['pit_id', 'depth_top', 'thickness', 'density_measured']],
    on=['pit_id', 'depth_top', 'thickness'],
    how='left'
).drop(columns=['depth_top', 'thickness'])  # Drop temporary columns after merge


Apply Köchle Elastic Modulus Parameterization

Now we'll apply the Köchle and Schneebeli (2014) method to calculate elastic modulus for all layers that have density values.


In [ ]:
# Apply Köchle Elastic Modulus Parameterization to both density methods

# Initialize results storage
results = {
    'kim_jamieson_table2': {
        'elastic_modulus_values': [],
        'successful_count': 0
    },
    'measured': {
        'elastic_modulus_values': [],
        'successful_count': 0
    }
}

# Process kim_jamieson_table2 density method (with uncertainty)
print("Processing kim_jamieson_table2 density method...")
for idx, row in layer_df.iterrows():
    if pd.notna(row['density_kim_jamieson_table2']) and pd.notna(row['density_kim_jamieson_table2_uncertainty']) and pd.notna(row['kim_jamieson_table2_grain_form']):
        # Create ufloat with density and uncertainty
        density_ufloat = ufloat(row['density_kim_jamieson_table2'], row['density_kim_jamieson_table2_uncertainty'])
        grain_form = row['kim_jamieson_table2_grain_form']
        
        # Calculate elastic modulus using Köchle method
        try:
            E_modulus = elastic_modulus.calculate_elastic_modulus(method='kochle', density=density_ufloat, grain_form=grain_form)
            
            # Check if result is valid (not NaN)
            if not np.isnan(E_modulus.nominal_value):
                results['kim_jamieson_table2']['elastic_modulus_values'].append(E_modulus)
                results['kim_jamieson_table2']['successful_count'] += 1
                
                # Store results back in dataframe
                layer_df.at[idx, 'elastic_modulus_kochle_kim_jamieson_table2'] = E_modulus.nominal_value
                layer_df.at[idx, 'elastic_modulus_kochle_kim_jamieson_table2_uncertainty'] = E_modulus.std_dev
                layer_df.at[idx, 'elastic_modulus_kochle_kim_jamieson_table2_relative_uncertainty'] = E_modulus.std_dev / E_modulus.nominal_value
            else:
                layer_df.at[idx, 'elastic_modulus_kochle_kim_jamieson_table2'] = np.nan
                layer_df.at[idx, 'elastic_modulus_kochle_kim_jamieson_table2_uncertainty'] = np.nan
                layer_df.at[idx, 'elastic_modulus_kochle_kim_jamieson_table2_relative_uncertainty'] = np.nan
                
        except Exception as e:
            layer_df.at[idx, 'elastic_modulus_kochle_kim_jamieson_table2'] = np.nan
            layer_df.at[idx, 'elastic_modulus_kochle_kim_jamieson_table2_uncertainty'] = np.nan
            layer_df.at[idx, 'elastic_modulus_kochle_kim_jamieson_table2_relative_uncertainty'] = np.nan
# Process measured density method (without uncertainty)
print("Processing measured density method...")
for idx, row in layer_df.iterrows():
    density_measured = row['density_measured']
    
    # Handle the case where density_measured might be an array or have multiple values
    if density_measured is not None:
        # Convert to scalar if it's an array
        if hasattr(density_measured, '__len__') and not isinstance(density_measured, str):
            if len(density_measured) > 0:
                density_measured = density_measured[0]
            else:
                density_measured = None
        
        # Check if we have a valid scalar value and grain form
        if density_measured is not None and pd.notna(density_measured) and pd.notna(row['kim_jamieson_table2_grain_form']):
            # Create ufloat with density but no uncertainty (std_dev = 0)
            density_ufloat = ufloat(float(density_measured), 0.0)
            grain_form = row['kim_jamieson_table2_grain_form']
        
            # Calculate elastic modulus using Köchle method
            try:
                E_modulus = elastic_modulus.calculate_elastic_modulus(method='kochle', density=density_ufloat, grain_form=grain_form)
                
                # Check if result is valid (not NaN)
                if not np.isnan(E_modulus.nominal_value):
                    results['measured']['elastic_modulus_values'].append(E_modulus)
                    results['measured']['successful_count'] += 1
                    
                    # Store results back in dataframe
                    layer_df.at[idx, 'elastic_modulus_kochle_measured'] = E_modulus.nominal_value
                    layer_df.at[idx, 'elastic_modulus_kochle_measured_uncertainty'] = E_modulus.std_dev
                    layer_df.at[idx, 'elastic_modulus_kochle_measured_relative_uncertainty'] = E_modulus.std_dev / E_modulus.nominal_value
                else:
                    layer_df.at[idx, 'elastic_modulus_kochle_measured'] = np.nan
                    layer_df.at[idx, 'elastic_modulus_kochle_measured_uncertainty'] = np.nan
                    
            except Exception as e:
                layer_df.at[idx, 'elastic_modulus_kochle_measured'] = np.nan
                layer_df.at[idx, 'elastic_modulus_kochle_measured_uncertainty'] = np.nan


Processing kim_jamieson_table2 density method...


Reload Module

Ensure we have the latest version of the elastic_modulus module with the umath.exp fix.


Test Köchle Method Implementation

First, let's verify the Köchle method works correctly with a test density value in the valid range.


In [ ]:
# Test the Köchle method with a sample density value
test_density = ufloat(200, 10)  # kg/m³, within valid range
test_grain_form = 'RG'  # Supported grain form for Köchle
test_E = elastic_modulus.calculate_elastic_modulus(method='kochle', density=test_density, grain_form=test_grain_form)
print(f"Test calculation: density = {test_density} kg/m³, grain_form = {test_grain_form}")
print(f"Elastic modulus = {test_E} MPa")
print(f"Is valid (not NaN): {not np.isnan(test_E.nominal_value)}")


In [ ]:

# Print summary statistics
print("\n=== KÖCHLE ELASTIC MODULUS CALCULATION RESULTS ===")
print("Kim-Jamieson Table 2 method:")
print(f"  - Total layers with kim_jamieson_table2 density: {layer_df['density_kim_jamieson_table2'].notna().sum()}")
print(f"  - Successful calculations: {results['kim_jamieson_table2']['successful_count']} ({(results['kim_jamieson_table2']['successful_count'] / layer_df['density_kim_jamieson_table2'].notna().sum()) * 100:.2f}%)")
if 'elastic_modulus_kochle_kim_jamieson_table2_relative_uncertainty' in layer_df.columns:
    print(f"  - Average relative uncertainty: {layer_df['elastic_modulus_kochle_kim_jamieson_table2_relative_uncertainty'].mean():.2f}%")
else:
    print(f"  - Average relative uncertainty: N/A (no successful calculations)")

print("\nMeasured density method:")
print(f"  - Total layers with measured density: {layer_df['density_measured'].notna().sum()}")
print(f"  - Successful calculations: {results['measured']['successful_count']} ({(results['measured']['successful_count'] / layer_df['density_measured'].notna().sum()) * 100:.2f}%)")
if 'elastic_modulus_kochle_measured_relative_uncertainty' in layer_df.columns:
    print(f"  - Average relative uncertainty: {layer_df['elastic_modulus_kochle_measured_relative_uncertainty'].mean():.2f}%")
else:
    print(f"  - Average relative uncertainty: N/A (no successful calculations)")

# Print diagnostic information about density ranges
print("\n=== DENSITY RANGE ANALYSIS ===")
print(f"Kim-Jamieson Table 2 density range: {layer_df['density_kim_jamieson_table2'].min():.1f} - {layer_df['density_kim_jamieson_table2'].max():.1f} kg/m³")
print(f"Measured density range: {layer_df['density_measured'].min():.1f} - {layer_df['density_measured'].max():.1f} kg/m³")
print(f"\nKöchle method valid range: 150 - 450 kg/m³")
print(f"Kim-Jamieson Table 2 layers in valid range: {((layer_df['density_kim_jamieson_table2'] >= 150) & (layer_df['density_kim_jamieson_table2'] <= 450)).sum()}")
print(f"Measured layers in valid range: {((layer_df['density_measured'] >= 150) & (layer_df['density_measured'] <= 450)).sum()}")

# Save updated dataframe
layer_df.to_csv('layer_df.csv', index=False)
print("\nUpdated layer data saved to layer_df.csv")
